<a href="https://colab.research.google.com/github/karunaprakash062/Major_Project/blob/main/featureselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('/content/drive/MyDrive/NSL-KDD/NSL_KDD_Data.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['service'] = encoder.fit_transform(data['service'])
data['protocol_type'] = encoder.fit_transform(data['protocol_type'])
data['flag'] = encoder.fit_transform(data['flag'])

In [5]:
removed_feature = data.pop('num_outbound_cmds')

In [6]:
target = data.pop('class')

Filter Methods for features selection

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif


X, y = data, target
feature_names = data.columns

# Chi-square test
chi2_selector = SelectKBest(chi2, k=10)
X_chi2_selected = chi2_selector.fit_transform(X, y)
selected_chi2_indices = chi2_selector.get_support(indices=True)
selected_chi2_features = [feature_names[i] for i in selected_chi2_indices]

# ANOVA F-value (f_classif)
fvalue_selector = SelectKBest(f_classif, k=10)
X_fvalue_selected = fvalue_selector.fit_transform(X, y)
selected_fvalue_indices = fvalue_selector.get_support(indices=True)
selected_fvalue_features = [feature_names[i] for i in selected_fvalue_indices]

# Mutual Information
mi_selector = SelectKBest(mutual_info_classif, k=10)
X_mi_selected = mi_selector.fit_transform(X, y)
selected_mi_indices = mi_selector.get_support(indices=True)
selected_mi_features = [feature_names[i] for i in selected_mi_indices]

print("Chi-square selected features:", selected_chi2_features)
print("ANOVA F-value selected features:", selected_fvalue_features)
print("Mutual Information selected features:", selected_mi_features)

Chi-square selected features: ['duration', 'service', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'hot', 'count', 'srv_count', 'dst_host_count', 'dst_host_srv_count']
ANOVA F-value selected features: ['flag', 'land', 'wrong_fragment', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'dst_host_same_srv_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate']
Mutual Information selected features: ['service', 'flag', 'src_bytes', 'dst_bytes', 'count', 'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate']


In [8]:
# Convert the selected features lists to sets
set_chi2 = set(selected_chi2_features)
set_fvalue = set(selected_fvalue_features)
set_mi = set(selected_mi_features)

# Find common features between Chi-square and ANOVA F-value
common_chi2_fvalue = set_chi2.intersection(set_fvalue)

# Find common features between Chi-square and Mutual Information
common_chi2_mi = set_chi2.intersection(set_mi)

# Find common features between ANOVA F-value and Mutual Information
common_fvalue_mi = set_fvalue.intersection(set_mi)

# Find common features present in any two sets among the three
common_any_two = common_chi2_fvalue.union(common_chi2_mi, common_fvalue_mi)

print("Common features present in any two sets among the three:", common_any_two)


Common features present in any two sets among the three: {'same_srv_rate', 'dst_host_srv_count', 'wrong_fragment', 'dst_host_same_srv_rate', 'count', 'service', 'src_bytes', 'dst_bytes', 'flag'}


Wrapper Methods for features selection